In [1]:
pip install prettytable==3.6.0 --upgrade

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install duckdb

# **PaySim Fraud Detection Analysis**

This notebook explores the PaySim dataset using SQL (DuckDB).  
Objective: uncover fraud patterns, detection gaps, and financial risks.

- Only ~0.2% of fraudulent transactions are flagged (`isFlaggedFraud`).
- This highlights a **major detection gap**: most fraudulent activity goes unnoticed.
- Strengthening monitoring for **large-value transfers and cash-outs** could drastically reduce undetected fraud.

In [1]:
import os
import pandas as pd
import duckdb

# Step 3: Set your dataset folder path
path_dir = r"C:\Users\COMPUTER POINT\Desktop\PaySim_Dataset"

# Step 4: Connect to a DuckDB database file (or use ':memory:' for temp session)
con = duckdb.connect("PaySim_new.db")  # file-based DB (you can open again later)

In [7]:
import os
os.listdir()

['.ipynb_checkpoints',
 'olist_ecommerce_paysim_new.db',
 'PaySim_Dataset.ipynb',
 'paysim_new.db',
 'Pay_Sim.csv']

In [9]:
duckdb.query("SELECT COUNT(*) AS total_rows FROM 'Pay_Sim.csv'").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,total_rows
0,6362620


In [2]:
for file in os.listdir(path_dir):
    if file.endswith(".csv"):
        file_path = os.path.join(path_dir, file)
        df = pd.read_csv(file_path)
        table_name = file.replace(".csv", "")
        con.register(table_name, df)  # creates temp table/view
        con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM {table_name}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
con.execute("SHOW TABLES").df()

,name
0,Pay_Sim
1,Pay_Sim


In [ ]:
con.execute(""" SELECT * FROM Pay_Sim LIMIT 10 """).df()

##  1.Transaction Overview

-Total transactions 
-Types of fraud transactions
-Types of non-fraudulent transactions
-Fraud 
-Average fraud vs legit size

Short insight after each query.

- Fraudulent transactions make up only a **tiny fraction** of overall transactions, but their amounts are disproportionately high.
- Fraud rate is below 1%, but the **financial exposure per fraud** is much larger compared to legitimate transactions.

In [5]:
con.execute(""" 
SELECT COUNT(*) AS Total_Transactions 
FROM Pay_Sim
""").df()

,Total_Transactions
0,6362620


In [36]:
con.execute("""
SELECT DISTINCT(type), CAST(SUM(amount) OVER (PARTITION BY type) AS BIGINT) AS non_Fraudalent_amount, 
COUNT(*) OVER (PARTITION BY type) AS num_transaction_nonfraud
FROM Pay_Sim 
WHERE isFraud = 0 
ORDER BY num_transaction_nonfraud
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,type,non_Fraudalent_amount,num_transaction_nonfraud
0,DEBIT,227199221,41432
1,TRANSFER,479224774079,528812
2,CASH_IN,236367391912,1399284
3,PAYMENT,28093371138,2151495
4,CASH_OUT,388423792981,2233384


In [15]:
con.execute("""
SELECT DISTINCT(type), CAST(SUM(amount) OVER (PARTITION BY type) AS BIGINT) AS Fraudalent_amount, COUNT(*) OVER (PARTITION BY type) AS num_transaction
FROM Pay_Sim
WHERE isFraud = 1

""").df()

,type,Fraudalent_amount,num_transaction
0,CASH_OUT,5989202244,4116
1,TRANSFER,6067213184,4097


In [20]:
con.execute("""
SELECT SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END)/ COUNT(*) AS Perc_Fraud_transaction
FROM Pay_Sim
 

""").df()


,Perc_Fraud_transaction
0,0.001291


In [25]:
con.execute("""
SELECT type, SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) /COUNT(*) as fraud_perc, CAST (SUM(amount) AS BIGINT) as amount_per_type
FROM Pay_Sim
GROUP BY type
ORDER BY fraud_perc DESC
""").df()

,type,fraud_perc,amount_per_type
0,TRANSFER,0.007688,485291987263
1,CASH_OUT,0.001840,394412995224
2,PAYMENT,0.000000,28093371138
3,CASH_IN,0.000000,236367391912
4,DEBIT,0.000000,227199221


In [41]:
con.execute("""
SELECT ROUND(AVG(amount),2) as AVG_AMOUNT, COUNT(*) AS NUM_TRANS, CASE WHEN isFraud = 1 THEN 'Fraud' ELSE 'Legitimate' END TRANS_TYPE
FROM Pay_Sim
GROUP BY IsFraud
""").df()


,AVG_AMOUNT,NUM_TRANS,TRANS_TYPE
0,1467967.30,8213,Fraud
1,178197.04,6354407,Legitimate


## **User-Level Analysis**

-Unique fraud users → good for scale of problem
-Fraud attempts from victims account (nameOrig) and type of transaction
-Fraud attempts from receivers account (nameDest) and type of transaction
-Repeated fraud attempts from receivers account (nameDest) → recidivism risk.
-Velocity analysis → very realistic fraud detection feature(Short term/ Long term/ High Burst)



- Most destination accounts are used only once, but a subset show **repeat fraud attempts**.
- Categorizing repeat frauds:
  - **High Burst**: multiple frauds within a day.
  - **Short Term**: repeat fraud within a week.
  - **Long Term**: repeat fraud across longer periods.
- Short-term repeat frauds dominate, showing fraudsters test accounts quickly rather than over months.

In [43]:
con.execute("""
SELECT COUNT(DISTINCT(nameOrig)) as fraud_users
FROM Pay_Sim
WHERE isFraud = 1
""").df()
-out of 6 billion accounts only 8213 were unique distinct users which tells fraud in not widespread or systematic it is concentrated 

,fraud_users
0,8213


In [26]:
con.execute("""
SELECT nameOrig, COUNT(*) AS num_trans, 
SUM(CASE WHEN type = 'TRANSFER' THEN amount ELSE 0 END) AS transfer_amount, 
SUM(CASE WHEN type = 'CASH_OUT' THEN amount ELSE 0 END) AS Cash_Out_amount
FROM Pay_Sim
WHERE isFraud = 1 
GROUP BY nameOrig
ORDER BY num_trans DESC
LIMIT 50
""").df()
-sender perspective no  victim account is used repeatedly for fraud. 

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,nameOrig,num_trans,transfer_amount,Cash_Out_amount
0,C1107458985,1,205129.12,0.00
1,C1954089665,1,15560.34,0.00
2,C1502031409,1,0.00,88748.63
3,C2031063966,1,0.00,623152.72
4,C1494570595,1,0.00,658637.38
5,C1919174675,1,343177.04,0.00
6,C55629100,1,0.00,343177.04
7,C1372492756,1,92145.40,0.00
8,C431054678,1,596127.11,0.00
9,C171367749,1,0.00,4677358.76


In [53]:
con.execute("""
SELECT nameDest, COUNT(*) as num_trans, 
SUM(CASE WHEN type = 'TRANSFER' THEN amount ELSE 0 END) AS transfer_amount, 
SUM(CASE WHEN type = 'CASH_OUT' THEN amount ELSE 0 END) AS Cash_Out_amount
FROM Pay_Sim
WHERE isFraud = 1 
GROUP BY nameDest
order by num_trans DESC
LIMIT 50
""").df()
#from the perspective of receivers there were multiple account used which identifies the fraud channel. 
#Destination accounts were used as intermediary in either transfer of funds or cash out 

,nameDest,num_trans,transfer_amount,Cash_Out_amount
0,C1148781766,2,449032.97,50523.78
1,C668046170,2,10000000.00,160088.68
2,C475338087,2,0.00,730733.52
3,C1185292292,2,10276.87,215299.28
4,C686334805,2,0.00,629142.78
5,C1399829166,2,0.00,3540009.28
6,C505532836,2,0.00,174238.21
7,C385133759,2,0.00,1396377.23
8,C803116137,2,0.00,1744765.69
9,C650699445,2,38580.45,369448.20


In [37]:
con.execute("""
With Fraud_counts AS (SELECT nameDest, COUNT(*) OVER (PARTITION BY nameDest) AS Fraud_counts
FROM Pay_Sim 
WHERE isFraud = 1
ORDER BY Fraud_counts DESC)
SELECT (SUM(CASE WHEN Fraud_counts > 1 THEN 1 ELSE 0 END)/ COUNT(*))*100 AS recidivism_risk,
COUNT(*) AS total_fraud_accounts,
SUM(CASE WHEN Fraud_counts > 1 THEN 1 ELSE 0 END) AS repeat_fraud_counts
FROM Fraud_counts
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,recidivism_risk,total_fraud_accounts,repeat_fraud_counts
0,1.071472,8213,88.0


In [27]:
con.execute("""
SELECT  nameDest, AVG(amount) AS avg_amount_receiver
FROM Pay_Sim
WHERE isFraud = 1 
GROUP BY  nameDest
ORDER BY avg_amount_receiver DESC
LIMIT 10
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,nameDest,avg_amount_receiver
0,C1908782637,10000000.0
1,C758711155,10000000.0
2,C19233081,10000000.0
3,C1973915590,10000000.0
4,C407821637,10000000.0
5,C1021550383,10000000.0
6,C1622860679,10000000.0
7,C428927923,10000000.0
8,C839316822,10000000.0
9,C1072055067,10000000.0


In [11]:
-velocity risk each step counts for 1 hour and total simulation is for 30 days

con.execute("""
SELECT DISTINCT(nameDest) , 
count(step) OVER (PARTITION BY nameDest) as num_step, 
MAX(step) OVER(PARTITION BY nameDest) as max_step , 
MIN(step) OVER (PARTITION BY nameDest) as min_step,
(MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) as step_diff,
CASE WHEN (MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) <= 24 THEN 'High Burst' 
     WHEN (MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) <= 168 THEN 'Short Term'
     ELSE 'Long Term' END AS recidivisim_risk_category

FROM Pay_Sim 
WHERE isFraud = 1
ORDER BY num_step DESC
limit 50


""").df()

 

,nameDest,num_step,max_step,min_step,step_diff,recidivisim_risk_category
0,C1399829166,2,314,212,102,Short Term
1,C1980653895,2,720,398,322,Long Term
2,C904300960,2,157,6,151,Short Term
3,C14138104,2,436,415,21,High Burst
4,C1827800010,2,724,368,356,Long Term
5,C1981613973,2,88,72,16,High Burst
6,C668046170,2,686,269,417,Long Term
7,C1827219533,2,95,36,59,Short Term
8,C964377943,2,401,300,101,Short Term
9,C1366192319,2,344,84,260,Long Term


In [23]:

con.execute("""
WITH step_count as  (SELECT DISTINCT(nameDest) , 
count(step) OVER (PARTITION BY nameDest) as num_step, 
MAX(step) OVER(PARTITION BY nameDest) as max_step , 
MIN(step) OVER (PARTITION BY nameDest) as min_step,
(MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) as step_diff,
CASE WHEN (MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) <= 24 THEN 'High Burst' 
     WHEN (MAX(step) OVER(PARTITION BY nameDest))-(MIN(step) OVER (PARTITION BY nameDest)) <= 168 THEN 'Short Term'
     ELSE 'Long Term' END AS recidivisim_risk_category

FROM Pay_Sim 
WHERE isFraud = 1
ORDER BY num_step DESC
limit 44)

SELECT recidivisim_risk_category, count(*)  as num_trans, count(*)/ (SELECT count(*) FROM step_count) as perc_per_category
FROM step_count
GROUP BY recidivisim_risk_category
ORDER BY perc_per_category DESC



""").df()

,recidivisim_risk_category,num_trans,perc_per_category
0,Short Term,18,0.409091
1,Long Term,17,0.386364
2,High Burst,9,0.204545


# **Transaction Size Patterns**
-Fraud distribution by transaction buckets and time buckets.
-Fraud rate higher in large-value transactions.


- Fraud is rare at small amounts but spikes at **higher transaction ranges**.
- Indicates fraudsters avoid small, suspicious activity and focus on **high-value single hits**.

In [39]:
con.execute("""
SELECT COUNT(amount) as num_transactions , 
CASE WHEN amount < 10000 THEN 'under  < 10k'
    WHEN amount < 100000 THEN '10k-100k' 
    ELSE 'greater than > 100K' END AS amount_bucket,
    CASE WHEN isFraud = 1 THEN 'Fraud' ELSE 'Legitimate' END AS account_type,
    
FROM Pay_Sim 
GROUP BY amount_bucket, account_type
ORDER BY num_transactions DESC, account_type ASC

""").df()




,num_transactions,amount_bucket,account_type
0,2830858,greater than > 100K,Legitimate
1,2237824,10k-100k,Legitimate
2,1285725,under < 10k,Legitimate
3,6506,greater than > 100K,Fraud
4,1429,10k-100k,Fraud
5,278,under < 10k,Fraud


In [43]:
con.execute("""
SELECT COUNT(amount) as num_transactions, 
CASE WHEN amount < 10000 THEN 'under  < 10k'
    WHEN amount < 100000 THEN '10k-100k' 
    ELSE 'greater than > 100K' END AS amount_bucket,
    COUNT(*)/(SELECT COUNT(*) FROM Pay_Sim WHERE isFraud = 1) as perc_fraud_trans
    
FROM Pay_Sim
WHERE isFraud = 1
GROUP BY amount_bucket
ORDER BY perc_fraud_trans DESC
""").df()

,num_transactions,amount_bucket,perc_fraud_trans
0,6506,greater than > 100K,0.792159
1,1429,10k-100k,0.173992
2,278,under < 10k,0.033849


In [48]:
con.execute("""
SELECT CASE 
    WHEN step % 24 BETWEEN 0 AND 6 THEN 'Night'
    WHEN step % 24 BETWEEN 7 AND 12 THEN 'Morning'
    WHEN step % 24 BETWEEN 13 AND 18 THEN 'Afternoon'
    ELSE 'Evening'
END AS hour_bucket, COUNT(*) as num_trans

FROM Pay_Sim
WHERE isFraud = 1
GROUP BY hour_bucket
ORDER BY num_trans DESC



""").df()

,hour_bucket,num_trans
0,Morning,2075
1,Afternoon,2081
2,Night,2354
3,Evening,1703


In [50]:
con.execute("""
SELECT
CASE 
    WHEN (step / 24) % 7 = 0 THEN 'Monday'
    WHEN (step / 24) % 7 = 1 THEN 'Tuesday'
    WHEN (step / 24) % 7 = 2 THEN 'Wednesday'
    WHEN (step / 24) % 7 = 3 THEN 'Thursday'
    WHEN (step / 24) % 7 = 4 THEN 'Friday'
    WHEN (step / 24) % 7 = 5 THEN 'Saturday'
    ELSE 'Sunday'
END AS day_of_week, count(*) as num_trans

FROM Pay_Sim
WHERE isFraud = 1
GROUP BY day_of_week
ORDER BY num_trans DESC
""").df()

,day_of_week,num_trans
0,Sunday,7953
1,Thursday,52
2,Saturday,50
3,Tuesday,44
4,Wednesday,42
5,Monday,38
6,Friday,34


# ** Detection Gap AND Fraud Impact**
-Top receivers by fraud amount or count
-Total amount lost to fraud
-Avg loss per event (detected fraud/ undetected fraud)
-Top 10 largest fraud cases



In [5]:
con.execute("""
SELECT nameDest, amount
FROM Pay_Sim
GROUP BY ROLLUP (nameDest, amount) 
ORDER BY amount DESC
LIMIT 10
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,nameDest,amount
0,C439737079,92445516.64
1,C753026640,73823490.36
2,C84111522,71172480.42
3,C167875008,69886731.30
4,C1472140329,69337316.27
5,C1757599079,67500761.29
6,C1073241084,66761272.21
7,C65111466,64234448.19
8,C514940761,63847992.58
9,C1871605747,63294839.63


In [26]:
con.execute("""
SELECT CAST ((SELECT SUM(amount) FROM Pay_Sim WHERE isFraud = 1 ) AS BIGINT) as total_amount_lost_to_fraud, 
(SELECT SUM(amount) FROM Pay_Sim WHERE isFraud = 1 )/SUM(amount)*100 as perc_fraud_to_total,
CAST((SELECT AVG(amount) FROM Pay_Sim WHERE isFraud = 1 ) AS BIGINT) as avg_loss_to_fraud_per_event

FROM Pay_Sim
""").df()

,total_amount_lost_to_fraud,perc_fraud_to_total,avg_loss_to_fraud_per_event
0,12056415428,1.053521,1467967


In [27]:
con.execute("""
SELECT SUM(CASE WHEN isFraud = 1 and isFlaggedFraud = 1 THEN 1 ELSE 0 END) AS detected_fraud, 
SUM(CASE WHEN isFraud = 1 and isFlaggedFraud = 0 THEN 1 ELSE 0 END) AS undetected_fraud

FROM Pay_Sim
""").df()

,detected_fraud,undetected_fraud
0,16.0,8197.0


In [29]:
con.execute("""
WITH fraud_count AS (SELECT SUM(CASE WHEN isFraud = 1 and isFlaggedFraud = 1 THEN 1 ELSE 0 END) AS detected_fraud, 
SUM(CASE WHEN isFraud = 1 and isFlaggedFraud = 0 THEN 1 ELSE 0 END) AS undetected_fraud
FROM Pay_Sim)
SELECT detected_fraud/(detected_fraud + undetected_fraud)*100 as detected_fraud_perc,
undetected_fraud/(detected_fraud + undetected_fraud)*100 as undetected_fraud_perc
FROM fraud_count
""").df()

,detected_fraud_perc,undetected_fraud_perc
0,0.194813,99.805187


In [34]:
con.execute("""
SELECT  CAST(AVG(CASE WHEN isFraud = 1 and isFlaggedFraud = 1 THEN AMOUNT ELSE NULL END) AS BIGINT) AS avg_detected_fraud,
CAST(AVG(CASE WHEN isFraud = 1 and isFlaggedFraud = 0 THEN AMOUNT ELSE NULL END) AS BIGINT) as avg_undetected_fraud
FROM Pay_Sim

""").df()

,avg_detected_fraud,avg_undetected_fraud
0,4861598,1461343
